# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
x_tfidf = tfidf_vect.fit_transform(data['body_text'])
x_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(x_tfidf.toarray())], axis=1)

#convert column names
tfidf_column_numbers = x_tfidf_feat.columns
tfidf_column_strings = [str(x) for x in tfidf_column_numbers]
x_tfidf_feat.columns = tfidf_column_strings

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
x_count = count_vect.fit_transform(data['body_text'])
x_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(x_count.toarray())], axis=1)


#convert column names
count_column_numbers = x_count_feat.columns
count_column_strings = [str(x) for x in count_column_numbers]
x_count_feat.columns = count_column_strings


x_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
#we are defining our grid with hyper parameters. Thus, no need to pass argument in RFClassifier

rf = RandomForestClassifier()

#make a dictionary of parameters to be used
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30,60,90,None]}

#create a GridSearchCV model
gs = GridSearchCV(rf, param, cv=5, n_jobs = -1)

#fit the model
gs_fit = gs.fit(x_tfidf_feat, data['label'])

#get the results and turn it into a dataframe. Then, sort the rows by mean_test_score
#use list dividing methods to get the top five
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,28.873036,4.054912,0.200002,0.011687,None,300,"{'max_depth': None, 'n_estimators': 300}",0.981149,0.975763,0.977538,0.968553,0.970350,0.974671,0.004636,1
8,36.364908,1.521163,0.312511,0.055022,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.977558,0.973046,0.968553,0.973046,0.974132,0.003575,2
10,20.198533,0.711210,0.284540,0.018053,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.974865,0.974843,0.967655,0.975741,0.973953,0.003219,3
7,18.701695,0.821263,0.287689,0.030817,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977558,0.975763,0.973944,0.965858,0.973046,0.973234,0.004001,4
6,2.437065,0.110357,0.171875,0.019764,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.973070,0.971275,0.974843,0.971249,0.972147,0.972517,0.001342,5


In [4]:
#now we want to run this also with the count vectorisation

rf = RandomForestClassifier()

param = {'n_estimators': [10, 150, 300],
        'max_depth': [30,60,90,None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs = -1)

gs_fit = gs.fit(x_count_feat, data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,18.956689,0.785649,0.328384,0.049336,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977558,0.976661,0.974843,0.969452,0.970350,0.973773,0.003292,1
8,36.783804,1.857238,0.340709,0.055449,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.975763,0.973046,0.967655,0.970350,0.973054,0.003820,2
11,29.685241,3.587655,0.224961,0.016348,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978456,0.975763,0.973944,0.967655,0.969452,0.973054,0.003986,2
10,20.957302,0.747026,0.284541,0.011540,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.971275,0.973944,0.967655,0.972147,0.972516,0.003249,4
6,2.688928,0.194544,0.200286,0.033433,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.972172,0.974865,0.973944,0.965858,0.968553,0.971079,0.003388,5
